# Lab 06: Spatial smoothing, denoising with ICA, and temporal filtering

The goal of this lab is to practice common pre-preprocessing steps that all involve some form of filtering out signal that is deemed to be "noise" based on signal processing concepts. We will look at three common steps that follow motion correction and slice-timing: 
* Spatial smoothing
* ICA-based denoising
* Temporal filtering

## Preparation

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from nilearn import plotting
from nilearn import image

In [4]:
%%bash

# DECIDE WHETHER YOU WANT TO COPY (cp) OR DOWNLOAD THE FILES NECESSARY FOR THIS LAB

# default
ans="cp"

if [ "${ans}" == "cp" ]; then
    cp ../05-Lab/my_bold_mcf_st.nii.gz my_bold_mcf_st.nii.gz
    cp ../05-Lab/my_bold_mcf.par my_bold_mcf.par
    cp ../04-Lab/func2T1.mat func2T1.mat
    cp ../04-Lab/T1_2_MNI_warp.nii.gz T1_2_MNI_warp.nii.gz
elif [ "${ans}" == "dl" ]; then

    # Link doesn't work yet place holder
    wget --quiet -O 06-Lab_data.tar.gz https://osf.io/j7t6n/download &&\
    tar -xf 06-Lab_data.tar.gz &&\
    rm 06-Lab_data.tar.gz
else
    printf 'YOU FORGOT TO CHANGE ANS TO "cp" OR "dl"'
fi

## Spatial smoothing in FSL

FSL's smoothing tool is called SUSAN
* See user manual [here](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/SUSAN)
* An important usage note is that SUSAN takes in the smoothing kernel as "sigma" which refers to the standard deviation of the smoothing kernel rather than the Full-Width at Half-Max (FWHM) in mm. 
    * [Link](https://www.jiscmail.ac.uk/cgi-bin/webadmin?A2=FSL;d7249c17.1301) to an informative post on the FSL listserv
    * Conversion from sigma from FWHM = `sigma*sqrt(8*ln(2)) = sigma*2.3548`
    * Conversion from FWHM from sigma = `mm/sqrt(8*ln(2))`

In [19]:
%%bash

# Usage 
susan


Usage: susan <input> <bt> <dt> <dim> <use_median> <n_usans> [<usan1> <bt1> [<usan2> <bt2>]] <output>
<bt> is brightness threshold and should be greater than noise level and less than contrast of edges to be preserved.
<dt> is spatial size (sigma, i.e., half-width) of smoothing, in mm.
<dim> is dimensionality (2 or 3), depending on whether smoothing is to be within-plane (2) or fully 3D (3).
<use_median> determines whether to use a local median filter in the cases where single-point noise is detected (0 or 1).
<n_usans> determines whether the smoothing area (USAN) is to be found from secondary images (0, 1 or 2).
A negative value for any brightness threshold will auto-set the threshold at 10% of the robust range


In [32]:
%%bash

# Strip the skull from the motion corrected and possibly slice time corrected image

# Take mean of the functional image over time
fslmaths my_bold_mcf_st.nii.gz -Tmean mean_func
# Bet the mean func, tell it to only generate the mask for now, then we rename mask_mask to mask
bet2 mean_func mask -f .3 -n -m; mv mask_mask.nii.gz mask.nii.gz
# Apply mask to create brain extracted functional image
fslmaths my_bold_mcf_st.nii.gz -mas mask.nii.gz my_bold_mcf_st_bet.nii.gz

In [34]:
%%bash

# Take a look
fslview my_bold_mcf_st_bet.nii.gz mask.nii.gz

In [45]:
%%bash

# FSL preprocessing steps to find brightness thresh and create liberal functional mask 

# Find "robust" range from 2nd to 98th percentile
robust_range=$(fslstats my_bold_mcf_st_bet.nii.gz -p 2 -p 98)
echo "Robust range is ${robust_range}"

# Take 10th percentile of this robust range and threshold functional image and make new mask
# Voxels most likely noise get excluded from this mask
fslmaths my_bold_mcf_st_bet.nii.gz -thr 111.3 -Tmin -bin mask -odt char

# Find the median within this mask and then 75% of the median within that mask 
brightness_thresh=$(fslstats my_bold_mcf_st.nii.gz -k mask -p 50 |  awk '{print ($1*0.75)}')
echo "Brightness thresh for epi is ${brightness_thresh}"

# Dilate the mask to give some extra buffer space
fslmaths mask -dilF mask

# Apply mask and create new mean func
fslmaths my_bold_mcf_st.nii.gz -mas mask.nii.gz my_bold_mcf_st_thresh.nii.gz
fslmaths my_bold_mcf_st_thresh.nii.gz -Tmean mean_func

Robust range is 0.000000 1113.125000 
Brightness thresh for epi is 635.909


In [46]:
%%bash

# Now we're ready to smooth!

# If working from the terminal, convert desired FWHM (mm) to sigma for input to SUSAN
# Below is an example of smoothing at 6mm FWHM
smooth=6
smoothSigma=$(echo $smooth | awk '{print ($1/(sqrt(8*log(2))))}')
echo "Smoothing kernel set to ${smooth}mm which is ${smoothSigma} in sigma"


Smoothing kernel set to 6mm which is 2.54797 in sigma


In [51]:
%%bash

# Call to SUSAN
# I had trouble calling variables above, wonder why?

susan my_bold_mcf_st_thresh.nii.gz 635.909 2.54797 3 1 1 mean_func.nii.gz 635.909 my_bold_mcf_st_thresh_smooth.nii.gz


In [52]:
%%bash

# Compare un-smoothed and smoothed functional image
fslview my_bold_mcf_st_thresh.nii.gz my_bold_mcf_st_thresh_smooth.nii.gz

## ICA-AROMA

* It's my understanding that median intensity scaling isn't necessary so we could simply use as inputs:
    * my_bold_mcf_st_thresh_smooth.nii.gz
    * .par file copied above for motion parameters
    * reg files copied above
    * create a more snug mask with our mean func as suggested in manual
* Should we have them download ica-aroma from github and run or just download output and understand? Any preference on clone vs. download when pulling a package from github that you won't contributed to. Cloning can make it easier to pull in updates?
* It would take too long to run during class and to run on a new data set could be their lab problem since they'd have to run from scratch

In [55]:
%%bash
git clone https://github.com/maartenmennes/ICA-AROMA.git

fatal: destination path 'ICA-AROMA' already exists and is not an empty directory.


In [57]:
%%bash

# Make a snugger mask
bet mean_func.nii.gz aroma_mask -f .3 -n -m

In [58]:
%%bash
fslview mean_func.nii.gz aroma_mask_mask.nii.gz

In [77]:
%%bash

# Call to ICA-AROMA
# Show how to call in bash since they would write in a shell script
# What am I doing wrong with below command?

python ICA-AROMA/ICA_AROMA.py -in my_bold_mcf_st_thresh_smooth.nii.gz -out ica_aroma -affmat func2T1.mat -warp T1_2_MNI_warp.nii.gz -mc my_bold_mcf.par -m aroma_mask_mask.nii.gz


------------------------------- RUNNING ICA-AROMA ------------------------------- 
--------------- 'ICA-based Automatic Removal Of Motion Artifacts' --------------- 

The specified input file does not exist.
The specified mc file does does not exist.
The specified affmat file does not exist.
The specified warp file does not exist.
The specified mask does not exist.

----------------------------- ICA-AROMA IS CANCELED -----------------------------



/home/brain/Labs/06-Lab
